# Forecasting Pipeline — BVMT Trading Assistant

This notebook demonstrates:
1. Data preparation for forecasting
2. SARIMA model fitting
3. XGBoost model fitting
4. Ensemble forecasting
5. Backtesting & evaluation
6. Visualization of results

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from modules.common.data_loader import load_all_data, get_stock_data, get_stock_list, add_technical_indicators
from modules.forecasting.forecaster import BVMTForecaster

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print('Modules loaded')

## 1. Data Loading & Preparation

In [ ]:
# Load all BVMT data
all_data = load_all_data(os.path.join('..', 'data'))
stocks = get_stock_list(all_data)
print(f"Loaded {len(all_data):,} records, {len(stocks)} stocks")

# Pick a liquid stock for demonstration
top_by_volume = all_data.groupby('code')['volume'].mean().sort_values(ascending=False)
STOCK = top_by_volume.index[0]
print(f"\nSelected stock: {STOCK} (avg daily volume: {top_by_volume.iloc[0]:,.0f})")

stock_df = get_stock_data(all_data, STOCK)
stock_df = add_technical_indicators(stock_df)
print(f"Data points: {len(stock_df)}")
print(f"Date range: {stock_df['date'].min()} to {stock_df['date'].max()}")

stock_df.tail()

In [ ]:
# Prepare time series
ts = stock_df.set_index('date')['close'].dropna().sort_index()
ts = ts[~ts.index.duplicated(keep='first')]

# Train/test split (80/20)
split_idx = int(len(ts) * 0.8)
train = ts[:split_idx]
test = ts[split_idx:]

print(f"Training: {len(train)} points ({train.index[0].date()} to {train.index[-1].date()})")
print(f"Testing:  {len(test)} points ({test.index[0].date()} to {test.index[-1].date()})")

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train.index, train.values, color='#58a6ff', label='Train')
ax.plot(test.index, test.values, color='#f0883e', label='Test')
ax.axvline(test.index[0], color='white', linestyle='--', alpha=0.5)
ax.set_title(f'{STOCK} — Train/Test Split', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

## 2. Stationarity Check

In [ ]:
forecaster = BVMTForecaster()

# Check stationarity of raw prices
stat_result = forecaster.check_stationarity(train)
print("=== Stationarity of Raw Prices ===")
print(f"ADF statistic: {stat_result['adf_statistic']:.4f}")
print(f"ADF p-value:   {stat_result['adf_pvalue']:.4f}")
print(f"KPSS statistic: {stat_result['kpss_statistic']:.4f}")
print(f"KPSS p-value:  {stat_result['kpss_pvalue']:.4f}")
print(f"Stationary:    {stat_result['is_stationary']}")

# Check stationarity of returns
returns = train.pct_change().dropna()
stat_ret = forecaster.check_stationarity(returns)
print("\n=== Stationarity of Returns ===")
print(f"ADF p-value:   {stat_ret['adf_pvalue']:.6f}")
print(f"KPSS p-value:  {stat_ret['kpss_pvalue']:.4f}")
print(f"Stationary:    {stat_ret['is_stationary']}")

## 3. SARIMA Model

In [ ]:
# Fit SARIMA with automatic order selection
print("Fitting SARIMA model (this may take 1-2 minutes)...")
sarima_result = forecaster.fit_sarima(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 5))

if sarima_result:
    print(f"SARIMA Order: {sarima_result['order']}")
    print(f"Seasonal Order: {sarima_result['seasonal_order']}")
    print(f"AIC: {sarima_result['aic']:.2f}")
    print(f"BIC: {sarima_result['bic']:.2f}")
    
    # Forecast on test period
    model = sarima_result['model']
    forecast_steps = min(len(test), 30)
    forecast = model.forecast(steps=forecast_steps)
    conf_int = model.get_forecast(steps=forecast_steps).conf_int()
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(train.index[-60:], train.values[-60:], color='#58a6ff', label='Train (last 60 days)')
    ax.plot(test.index[:forecast_steps], test.values[:forecast_steps], color='#f0883e', label='Actual')
    ax.plot(test.index[:forecast_steps], forecast.values, color='#3fb950', linewidth=2, label='SARIMA Forecast')
    ax.fill_between(test.index[:forecast_steps],
                    conf_int.iloc[:, 0], conf_int.iloc[:, 1],
                    color='#3fb950', alpha=0.15, label='95% CI')
    ax.set_title(f'{STOCK} — SARIMA Forecast vs Actual', fontsize=14)
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(test.values[:forecast_steps], forecast.values)
    rmse = np.sqrt(mean_squared_error(test.values[:forecast_steps], forecast.values))
    mape = np.mean(np.abs((test.values[:forecast_steps] - forecast.values) / test.values[:forecast_steps])) * 100
    print(f"\nSARIMA Performance ({forecast_steps}-day):")
    print(f"  MAE:  {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
else:
    print("SARIMA fitting failed — check data quality")

## 4. XGBoost Model

In [ ]:
# Fit XGBoost
print("Fitting XGBoost model...")
xgb_result = forecaster.fit_xgboost(train, n_lags=10)

if xgb_result:
    print(f"Training R²: {xgb_result['train_score']:.4f}")
    
    # Feature importance
    fi = xgb_result['feature_importance']
    top_features = sorted(fi.items(), key=lambda x: x[1], reverse=True)[:10]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Feature importance
    names, values = zip(*top_features)
    axes[0].barh(range(len(names)), values, color='#58a6ff')
    axes[0].set_yticks(range(len(names)))
    axes[0].set_yticklabels(names)
    axes[0].set_title('XGBoost Feature Importance', fontsize=14)
    axes[0].invert_yaxis()
    
    # Rolling prediction on test
    predictions = []
    forecast_steps = min(len(test), 30)
    history = list(train.values)
    
    model = xgb_result['model']
    n_lags = 10
    
    for i in range(forecast_steps):
        features = np.array(history[-n_lags:]).reshape(1, -1)
        pred = model.predict(features)[0]
        predictions.append(pred)
        history.append(test.values[i])  # use actual for next step
    
    axes[1].plot(test.index[:forecast_steps], test.values[:forecast_steps],
                 color='#f0883e', label='Actual')
    axes[1].plot(test.index[:forecast_steps], predictions,
                 color='#3fb950', linewidth=2, label='XGBoost')
    axes[1].set_title(f'{STOCK} — XGBoost Forecast', fontsize=14)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    mae = mean_absolute_error(test.values[:forecast_steps], predictions)
    rmse = np.sqrt(mean_squared_error(test.values[:forecast_steps], predictions))
    mape = np.mean(np.abs((test.values[:forecast_steps] - np.array(predictions)) / test.values[:forecast_steps])) * 100
    print(f"\nXGBoost Performance ({forecast_steps}-day):")
    print(f"  MAE:  {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")

## 5. Full Forecasting Pipeline

In [ ]:
# Run the full forecast pipeline
print(f"Running full forecast pipeline for {STOCK}...")
result = forecaster.forecast(train, horizon=10)

if result:
    print(f"\nModels fitted: {result.get('models_fitted', [])}")
    print(f"Forecast horizon: {len(result.get('forecast', []))} days")
    
    fc = result['forecast']
    ci_lower = result.get('conf_int_lower', [])
    ci_upper = result.get('conf_int_upper', [])
    
    # Generate forecast dates (business days)
    last_date = train.index[-1]
    forecast_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=len(fc))
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(train.index[-30:], train.values[-30:], color='#58a6ff', linewidth=2, label='Historical')
    ax.plot(forecast_dates, fc, color='#3fb950', linewidth=2, marker='o', markersize=4, label='Ensemble Forecast')
    
    if ci_lower and ci_upper:
        ax.fill_between(forecast_dates, ci_lower, ci_upper,
                        color='#3fb950', alpha=0.15, label='95% CI')
    
    # Mark actual if available
    actual_overlap = test[:len(fc)]
    if len(actual_overlap) > 0:
        ax.plot(actual_overlap.index[:len(fc)], actual_overlap.values[:len(fc)],
                color='#f0883e', linewidth=2, marker='s', markersize=4, label='Actual')
    
    ax.set_title(f'{STOCK} — Ensemble Forecast (10-day)', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print forecast table
    print("\nDay-by-day forecast:")
    for i, (date, val) in enumerate(zip(forecast_dates, fc)):
        lo = ci_lower[i] if ci_lower else 'N/A'
        hi = ci_upper[i] if ci_upper else 'N/A'
        print(f"  {date.strftime('%Y-%m-%d')}: {val:.3f}  [{lo if isinstance(lo, str) else f'{lo:.3f}'} — {hi if isinstance(hi, str) else f'{hi:.3f}'}]")

## 6. Backtesting

In [ ]:
# Walk-forward backtest
print("Running walk-forward backtest (5-day windows)...")
backtest = forecaster.evaluate_backtest(ts, test_size=0.2, horizon=5)

if backtest:
    print("\n=== Backtest Results ===")
    for metric, value in backtest.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.4f}")
        else:
            print(f"  {metric}: {value}")

## 7. Multi-Stock Comparison

In [ ]:
# Compare forecast accuracy across top 5 stocks
top5 = top_by_volume.index[:5].tolist()
comparison = []

for code in top5:
    print(f"\nForecasting {code}...")
    s_df = get_stock_data(all_data, code)
    s_ts = s_df.set_index('date')['close'].dropna().sort_index()
    s_ts = s_ts[~s_ts.index.duplicated(keep='first')]
    
    if len(s_ts) < 100:
        print(f"  Skipping {code} — insufficient data ({len(s_ts)} points)")
        continue
    
    s_split = int(len(s_ts) * 0.8)
    s_train = s_ts[:s_split]
    s_test = s_ts[s_split:]
    
    fc_obj = BVMTForecaster()
    result = fc_obj.forecast(s_train, horizon=5)
    
    if result and len(s_test) >= 5:
        fc_vals = np.array(result['forecast'][:5])
        actual = s_test.values[:5]
        mae = np.mean(np.abs(actual - fc_vals))
        mape = np.mean(np.abs((actual - fc_vals) / actual)) * 100
        direction = np.mean(np.sign(np.diff(actual)) == np.sign(np.diff(fc_vals))) * 100
        comparison.append({'Stock': code, 'MAE': mae, 'MAPE': mape, 'Dir_Acc': direction})
        print(f"  MAE={mae:.2f}, MAPE={mape:.1f}%, DirAcc={direction:.0f}%")

if comparison:
    comp_df = pd.DataFrame(comparison)
    print("\n=== Forecast Accuracy Comparison ===")
    display(comp_df)

## Summary

- **SARIMA** provides a solid statistical baseline with interpretable confidence intervals
- **XGBoost** captures nonlinear patterns via lag features and technical indicators
- **Ensemble** combines both for more robust predictions
- Walk-forward backtesting validates out-of-sample performance
- Forecast accuracy varies by stock liquidity — more liquid stocks tend to be more predictable